# Extracting and pre-processing the transcription's text from the PDF files

In this notebook, we extract the text from the PDF files, pre-process it, and generate some statistics.

## Installing Libraries

In [ ]:
!pip install -U  pdfplumber

## Importando bibliotecas

In [ ]:
import pdfplumber
import os
import re
import string
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

nltk.download('stopwords')
nltk.download('punkt')


In [ ]:
folder = './'

## Funções

In [ ]:
# As funções abaixo foram adaptadas de: https://github.com/jsvine/pdfplumber/issues/356#issuecomment-1471361607

# Retorna se um objeto não está contido em outro
# Por exemplo: se um texto está contido em uma tabela ou figura
def not_within_bboxes(obj,bboxes):

    def obj_in_bbox(_bbox):
        v_mid = (obj["top"] + obj["bottom"]) / 2
        h_mid = (obj["x0"] + obj["x1"]) / 2
        x0, top, x1, bottom = _bbox
        return (h_mid >= x0) and (h_mid < x1) and (v_mid >= top) and (v_mid < bottom)

    return not any(obj_in_bbox(__bbox) for __bbox in bboxes)

def curves_to_edges(cs):
    edges = []
    for c in cs:
        edges += pdfplumber.utils.rect_to_edges(c)
    return edges

# Extrai o texto de um arquivo PDF que não está dentro de tabelas ou figuras
def raw_text_extract(pdf_file, include_tables=False, include_images=False, show_page_number=False):
    page_data = []
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            #page = pdf.pages[0]

            if show_page_number:
                print(f"Pagina: {page.page_number}")

            #print(page.extract_text())
            bboxes = []
            # identificando as tabelas
            if not include_tables:
                bboxes = [
                    table.bbox
                    for table in page.find_tables(
                    table_settings={
                        "vertical_strategy": "lines",
                        "horizontal_strategy": "lines",
                        "explicit_vertical_lines": curves_to_edges(page.curves) + page.edges,
                        "explicit_horizontal_lines": curves_to_edges(page.curves) + page.edges,
                    }
                    )
                ]
            #print(bboxes)

            # identificando as imagens
            if not include_images:
                for image in page.images:
                    image_bbox = (image['x0'], image['top'], image['x1'], image['bottom'])
                    bboxes.append(image_bbox)
                    #print("img: ",image_bbox)

            # Filtrando os textos que estão fora das caixas das tabelas e das figuras
            page = page.filter(lambda obj: not_within_bboxes(obj, bboxes))

            # a arquivo bbdc-2015-3T-Transcrição da Teleconferência 3T15.pdf gera caracteres duplicados. Esse método resolve o problema.
            # ref: https://github.com/jsvine/pdfplumber/issues/71
            text = page.dedupe_chars().extract_text()


            ##### removendo cabeçalho: bbdc, bbas, prbc -> "Transc... ano": Pode ter ou não "Transcrição da"; Pode ter 3 ou 4 linhas

            # Transcrição da Teleconferência
            # Resultados do 4T09
            # Banco do Brasil (BBAS3 BZ) <- esta linha pode nao aparecer no BBDC
            # 26 de fevereiro de 2010

            # prbc-2013-2T13
            # Teleconferência do Paraná Banco
            # Resultados do 2° trimestre de 2013
            # 14 de junho de 2013 – 11h00 (horário de Brasília)
            text = re.sub(r"(Transcrição da )?Teleconferência(.*)?((?:\n|\r\n?)(.*))?((?:\n|\r\n?)(.*))?(?:\n|\r\n?)(.*)[0-9]{4}( – [0-9]{2}h(.*))?", '', text).strip()

            ##### Removendo cabeçalhos do ITUB
            # Itaú Unibanco
            # Resultados do Terceiro trimestre de 2018
            # 30 de outubro de 2018
            text = re.sub(r"Itaú Unibanco(.*)?((?:\n|\r\n?)(.*))?((?:\n|\r\n?)(.*))?(?:\n|\r\n?)(.*)[0-9]{4}", '', text).strip()

            ##### Removendo cabeçalhos do BBAS
            # - Palavra "#Pública" (3T19 a 3T22) e #interna (1T21 a 1T22)
            text = re.sub(f"#Pública", '', text).strip()

            # - Palavra "#interna" (1T21 a 1T22)
            text = re.sub(f"#interna", '', text).strip()


            # bbas-2006-4T06-Transcrição
            # Local Conference Call
            # Banco do Brasil Nac. – (29314)
            # Resultados do Exercício de 2006
            # 28 de Fevereiro de 2007 – 11:00h - horário local
            text = re.sub(r"Local Conference Call(?:\n|\r\n?)(.*)?((?:\n|\r\n?)(.*))?(?:\n|\r\n?)(.*)", '', text).strip()

            # bbas-2009-4T09-Transcrição.pdf
            # O padrão é este. As linhas iniciais são removidas na regra anterior, mas fica a linha da data, que é removida aqui
            # Transcrição da Teleconferência
            # Resultados do 4T09
            # Banco do Brasil (BBAS3 BZ)
            # 26 de fevereiro de 2010
            text = re.sub(r"^[0-9]{2}(.*)[0-9]{4}", '', text).strip()

            # Relação com Investidores
            # Transcrição 1T21
            # BANCO DO BRASIL
            # TELECONFERÊNCIA
            # DE RESULTADOS
            # 1T2021
            text = re.sub(r"Relação com Investidores(?:\n|\r\n?)(.*)?((?:\n|\r\n?)(.*))?((?:\n|\r\n?)(.*))?(?:\n|\r\n?)(.*)?(?:\n|\r\n?)(.*)", '', text).strip()

            # bbas-2020-3T20-Transcrição Teleconferência 3T20.pdf
            # BANCO DO BRASIL
            # TELECONFERÊNCIA
            # DE RESULTADOS
            # 3T2020
            # 06/11/2020
            text = re.sub(r"BANCO DO BRASIL(?:\n|\r\n?)(.*)?((?:\n|\r\n?)(.*))?((?:\n|\r\n?)(.*))?(?:\n|\r\n?)(.*)", '', text).strip()


            # removendo cabeçalho: abcb -> "Banco ABC Brasil | Relações com Investidores Transcrição da"
            text = re.sub(r"Banco ABC Brasil \| (.*)((?:\n|\r\n?))", '', text).strip()


            # removendo o número da página que veio junto do texto
            text = re.sub(f"\\n{page.page_number}$", '', text)

            text = re.sub(f'- {page.page_number} -', '', text)


            # removendo o número da página que veio junto do texto: bbdc -> "(cid:1) <pagina>"
            text = re.sub(f"\(cid:1\) {page.page_number}$", '', text).strip()

            # removendo o número da página que veio junto do texto: prbc -> "Página <pagina>"
            text = re.sub(f"Página {page.page_number}$", '', text).strip()

            # removendo o número da página que veio junto do texto: itub -> "1/12"
            text = re.sub(f'{page.page_number}/{len(pdf.pages)}$', '', text).strip()

            # removendo o número da página que veio junto do texto: itub -> "Teleconferência 4T21 2"
            text = re.sub(r'Teleconferência \dT\d{2} \d+$', '', text).strip()

            # removendo o número da página que veio junto do texto:
            # No abcp-2009-3T09, todas as páginas estão com número "9" e
            # no abcp-2009-3T09, todas as páginas estão com número 7
            head, tail = os.path.split(pdf_file)
            if (tail.startswith('abcb-2009')):
                if (tail.startswith('abcb-2009-3T09')):
                    text = re.sub(r'((?:\n|\r\n?))9$', '', text).strip()
                elif (tail.startswith('abcb-2009-4T09')):
                    text = re.sub(r'((?:\n|\r\n?))7$', '', text).strip()

            page_data.append(text)

    return page_data


In [ ]:
def clean(text):

    text = text.replace("\n", ' ')
    text = text.replace("”", '')
    text = text.replace("“", '')
    text = text.replace("\"", '')
    text = text.replace("", '')

    #lista com bolinha
    text = re.sub(r'(\s)?(;\s)?(•)', "; ", text.strip())
    text = re.sub(r'(: ;)', ": ", text.strip())
    text = re.sub(r'(\.;)', ". ", text.strip())

    # nu-2021-4T21-Script 4T21.pdf
    text = re.sub(r'(; -)', ". ", text.strip())
    text = re.sub(r'(\. -)', ". ", text.strip())
    text = re.sub(r'^(-)', "", text.strip())
    text = re.sub(r'(: \d.)', ': .', text.strip())
    text = re.sub(r'(; e (\d+\.)?)', '.', text.strip())
    text = re.sub(r'(: ●)', '.', text.strip())
    text = re.sub(r'(; ●)', '.', text.strip())
    text = re.sub(r'(●)', '', text.strip())

    text = re.sub("_______________________________________________________________", "", text)

    text = re.sub(r"Sra\.", "Senhora ", text, flags=re.IGNORECASE)
    text = re.sub(r"Sr\.", "Senhor ", text, flags=re.IGNORECASE)
    text = re.sub(r"Srs\.", "Senhores ", text, flags=re.IGNORECASE)
    text = re.sub(r'b\.p\.\s([A-Z])', 'bp. \\1', text).strip()
    text = re.sub('b.p.', 'bp', text).strip()
    text = re.sub('p.p.', 'pp', text).strip()
    text = re.sub('help!', 'help', text).strip() # bmgb -> tirar a exclamação para evitar quebra de sentenças

    text = re.sub('\s+', ' ', text).strip() # deixar por ultimo, pois as substituicoes anteriores podem inserir multiplos espaços

    return text

In [ ]:
#text = '-Tivemos um lucro líquido de R$ 70,8 MM, ou seja 11% superior quando comparado com 1S12; -O retorno sobre o capital médio neste semestre foi de 11,8%; -Volume de 957 milhões de originação de crédito consignado neste primeiro semestre, número 26,6% superior, comparado ao 1º semestre 2012; -A carteira de crédito ampliada(considerando avais e fiança) fechou o semestre em R$ 2.648,MM'
#text='-A carteira de crédito ampliada(considerando avais e fiança) fechou o semestre em R$ 2.648,MM com um crescimento de 18% sendo que a carteira de middle cresceu expressivos 44% nos últimos 12 meses; -96,5% da carteira de crédito entre níveis AA e C, indicando manutenção da qualidade da carteira;'
#text = "poderá obtê-lo no site do Banco - www.paranabanco.com.br/ri."
#text='internamente, como nossos cartões de crédito, nossos empréstimos pessoais e nossas soluções de pagamento por celular; e 2. Soluções de terceiros fornecidas por parcei'
text='internamente, como nossos cartões de crédito, nossos empréstimos pessoais e nossas soluções de pagamento por celular; e Soluções de terceiros fornecidas por parcei'
clean(text)

In [ ]:

def get_sentences(text):
    sentences = sent_tokenize(text, language='portuguese')
    sentences = [s for s in sentences if len(s.strip()) > 2]
    return sentences

def get_tokens(text):
    tokens = word_tokenize(text, language='portuguese')

    return tokens

def get_words(text):
    tokens = get_tokens(text)

    words = [w for w in tokens if w not in string.punctuation]

    return words

In [ ]:
def save_text(filename, sentences):
    with open(filename, 'w') as f:
        for row in sentences:
            if row != '':
                f.write(row+'\n')

In [ ]:
def generate_stats(text):
    stats = {}
    tokens = get_tokens(text)
    num_tokens = len(tokens)

    stats["tokens"] = num_tokens

    return stats

## Extraindo e pré-processando textos

In [ ]:
transcript_folder = folder+"pdf-files/"
transcript_text_folder = folder+"transcricoes_processadas/"

if not os.path.exists(transcript_text_folder):
    os.mkdir(transcript_text_folder)

global_stats = pd.DataFrame()

#Recupera os arquivos que estão na pasta
files = os.listdir(transcript_folder)

#files = [ f for f in files if f.startswith("nu") ]

#files = ['nu-2023-1T23-Transcrição  1T23.pdf']#'abcb-2009-4T09-Transcrição da Teleconferência 4T09.pdf']#'nu-2021-4T21-Script 4T21.pdf']#'prbc-2013-2T13-Transcrição 2T13.pdf']#'itub-2018-3T18-transcrição do áudio.pdf']#'bbdc-2015-2T-Transcrição da Teleconferência 2T15.pdf']#'bbdc-2018-1T-Transcrição da Teleconferência 1T18.pdf']#'bbas-2006-4T06-Transcrição.pdf']#'bbas-2009-4T09-Transcrição.pdf']#'bbas-2020-3T20-Transcrição Teleconferência 3T20.pdf'] #'bbas-2021-1T21-Transcrição 1T21.pdf']#'bbas-2019-3T19-Transcrição Áudio Tele 2T19.pdf']

num_files = len(files)
for i, file in enumerate(files):
    path_transcription = transcript_folder + file

    file_name_parts = file.split("-")
    ticker          = file_name_parts[0].strip()
    trimestre       = file_name_parts[2].strip()

    path_folder_ticker = os.path.join(transcript_text_folder, ticker)
    if not os.path.exists(path_folder_ticker):
        os.mkdir(path_folder_ticker)

    if os.path.isfile(path_transcription):
        print(f"Processando arquivo {i+1} de {num_files}")

        print("*** Extraindo texto do PDF: "+ file)
        path_transcription = transcript_folder + file
        if ticker in ['bbas', 'bbdc']:
            page_data = raw_text_extract(path_transcription)
        else:
            page_data = raw_text_extract(path_transcription, include_tables=True, include_images=False)

        #print(page_data)
        print("*** Limpando o texto")
        text = " ".join(page_data)
        text = clean(text)
        #print(text)

        print("*** Gerando sentenças")
        sentences = get_sentences(text)
        num_sentences = len(sentences)
        sentences = ['ticker;ano;trimestre;sentenca']+[f'{ticker};{file_name_parts[1]};{trimestre};"{s}"' for s in sentences]

        #[ print(s) for s in sentences ]

        print("*** Gerando estatísticas")

        stats = {}
        stats['origem']    = ticker
        stats['trimestre'] = trimestre
        stats['sentencas'] = num_sentences

        # concatenando os dois dicts
        stats = {**stats, **generate_stats(text)}

        print("*** Salvando arquivo com a sentenças")
        save_text(f"{path_folder_ticker}/{ticker}-{file_name_parts[1]+'-'+trimestre}.csv", sentences)

        global_stats = pd.concat([global_stats, pd.DataFrame.from_dict([stats])])


In [ ]:
global_stats

In [ ]:
global_stats.to_csv(transcript_text_folder + '/global_stats.csv', sep=',', index=False)

In [ ]:
# reunindo todos os arquivos em um só
dataset = transcript_text_folder+'/dataset-completo.csv'

print(f"Transcripts folder: {transcript_text_folder}")
dirs = os.listdir(transcript_text_folder)

dirs = [ d for d in dirs if os.path.isdir(os.path.join(transcript_text_folder, d)) ]
print(dirs)
#dirs = ['nu']
num_dirs = len(dirs)
print(f"Num dirs: {num_dirs}")
with open(dataset, 'w') as ds:
    ds.write(f"ticker;ano;trimestre;sentenca\n")
    for i, dir in enumerate(dirs):
        if dir != 'processadas':
            print(f"Dir: {dir} ({i+1}/{num_dirs})")
            path_transcription = os.path.join(transcript_text_folder, dir)

            files = os.listdir(path_transcription)
            num_files = len(files)
            for f_num, filename in enumerate(files):
                print(f"\tFile: {filename} ({f_num+1}/{num_files})")
                path_file = os.path.join(path_transcription, filename)

                if os.path.isfile(path_file):
                    with open(path_file) as f:
                        #pula a linha do cabeçalho dos arquivos
                        ds.writelines(f.readlines()[1:])

## Análise dos dados extraídos

In [ ]:
dataset = transcript_text_folder +'/dataset-completo.csv'

df = pd.read_csv(dataset, sep=';')
df


In [ ]:
# adicionando contagem de palavras
df['num_palavras'] = [ len(get_words(s)) for s in df['sentenca'].values]
df

In [ ]:
df.to_csv(f"{transcript_text_folder}/dados-completos-com-num-palavras.csv", sep=';', index=False)

In [ ]:
df[df['num_palavras'] < 5]

In [ ]:
df[df.duplicated(subset=['sentenca'])].sort_values(by='sentenca')

In [ ]:
print(f"Quantidade de sentenças: {len(df)}")

df_duplicatas = df[df.duplicated(subset=['sentenca'])].sort_values(by='sentenca')
df_duplicatas.to_csv(f"{transcript_text_folder}/dataset-sentencas-duplicadas.csv", sep=';', index=False)
print(f"Quantidade de duplicatas: {len(df_duplicatas)}")

df_sem_duplicatas = df.drop_duplicates(subset=['sentenca'])
df_sem_duplicatas.to_csv(f"{transcript_text_folder}/dataset-sem-sentencas-duplicadas.csv", sep=';', index=False)
print(f"Quantidade de sentenças sem duplicatas: {len(df_sem_duplicatas)}")

df_maiores_que_4 = df_sem_duplicatas[df_sem_duplicatas['num_palavras'] > 4]
df_maiores_que_4.to_csv(f"{transcript_text_folder}/dataset-com-sentencas-maiores-que-4-palavras.csv", sep=';', index=False)
print(f"Quantidade de sentenças com mais de 4 palavras: {len(df_maiores_que_4)}")

df_menores_que_5 = df_sem_duplicatas[df_sem_duplicatas['num_palavras'] < 5]
df_menores_que_5.to_csv(f"{transcript_text_folder}/dataset-com-sentencas-menores-que-5-palavras.csv", sep=';', index=False)
print(f"Quantidade de sentenças com menos de 5 palavras: {len(df_menores_que_5)}")

In [ ]:
df_maiores_que_4.hist(column='num_palavras', by='ano', sharex=True, figsize=(10,8));

In [ ]:
df_maiores_que_4.describe()

In [ ]:
df_maiores_que_4.groupby(by='ano').agg({'num_palavras':('min','max','mean', 'std')})

In [ ]:
df_maiores_que_4[df_maiores_que_4['num_palavras'] > 250]